# Define scraping functions

In [28]:
import requests
import time
import re
from bs4 import BeautifulSoup


def fetch_finn_codes(session: requests.Session, first=1, last=5):
    X_FINNKODE = "//*[@id='page-results']/div[1]/div/div/div/div[2]/div/a"
    URL_RESULTS = 'http://m.finn.no/car/used/search.html'
    codes = []
    for i in range(first, last):
        print("Page {}/{}".format(i, last))
        result = session.get(URL_RESULTS, params={'page': i})
        page = result.text
        doc = BeautifulSoup(page, 'lxml')
        ad_ids = [
            element.get('id')
            for element in doc.find_all('a')
            if element.get('class') and 'userhistory' in element.get('class')
        ]
        codes += ad_ids
    return codes


def fetch_ads(session, finn_codes, dt=0.05):
    URL_FETCH_AD = 'http://m.finn.no/car/used/ad.html?finnkode='
    ads = []
    for i, finn_code in enumerate(finn_codes):
        if not (i % (len(finn_codes)/100)):
            print("Progress: {}/{}".format(i, len(finn_codes)))
        time.sleep(dt)
        print(finn_code)
        try:
            test_ad = fetch_finn_code_url(session, URL_FETCH_AD + finn_code)
            test_ad['id'] = finn_code
            ads.append(test_ad)
        except ConnectionError as e:
            print('Connection error, sleeping...')
            sleep(10)
            print('Continuing')
        except Exception as e:
            print(e)
    return ads


def fetch_finn_code_url(session: requests.Session, finn_url):
    soup_page = BeautifulSoup(session.get(finn_url).text, 'lxml')
    data = {}
    # Handle tabled data
    kvps = {}
    data_dl_elements = soup_page.find_all('dl', attrs={'class': 'r-prl mhn multicol col-count1upto640 col-count2upto768 col-count1upto990 col-count2from990'})
    for elem in data_dl_elements:
        for key, value in zip(elem.find_all('dt'), elem.find_all('dd')):
            kvps[key.contents[0]] = value.contents[0]
    data['tabled'] = kvps
    
    ## Other data
    # Price
    soup_price = soup_page.findAll('div', {'class': 'h1 mtn r-margin', 'data-automation-id': 'value'})
    price = re.sub(r"\D", "", soup_price[0].contents[0])
    data['price'] = price
    
    # Brand
    sp_title = soup_page.findAll('h1', {'class': 'h1 word-break mbn'})
    title = sp_title[0].contents[0]
    data['title'] = title

    # Where
    soup_place = soup_page.findAll('h2')
    for p in soup_place:
        cmp = re.findall(r'\d\d\d\d \w+', p.contents[0])
        if cmp:
            place = cmp[0]
            break
    else:
        place = ''
    data['place'] = place
    
    return data

# Post-processing

In [2]:
import collections

# Order: flatten > strip_items

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def strip_items(flat_ads):
    stripped = []
    for d in flat_ads:
        stripped.append({
            key: value.strip().replace('\n', ' ')
            for key, value in d.items()
        })
    return stripped


# Handle CSV

In [3]:
import csv
import copy
import datetime

def export_data(filename, data):
    fields = set((key for d in data for key in d.keys()))
    with open(filename, 'w', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fields, lineterminator='\n')
        writer.writeheader()
        for d in data:
            writer.writerow(d)

# Processing

DATETIME_FORMATS = ['%d.%m.%Y', '%m.%Y', '%m %Y', '%Y', '%m/%Y']

def standardize_row(row):
    row_cp = copy.deepcopy(row)
    for key, value in row.items():
        if not value:
            continue
        if '1. gang registrert' in key:
            for date_format in DATETIME_FORMATS:
                try:
                    t = datetime.datetime.strptime(value, date_format)  #'20.11.2011'
                except ValueError:
                    continue
                break
            else:
                print('Could not parse date: %s' % value)
                continue
            row_cp[key] = t.strftime(DATETIME_FORMATS[0])
        elif key in ['tabled_Pris eks omreg', 'tabled_Omregistrering']:
            try:
                row_cp[key] = int(re.sub(r"\D", "", value))
            except ValueError:
                print('Could not parse number: %s' % value)
    return row_cp

def standardize_csv(ifn, ofn):
    with open(ifn, encoding='utf-8') as ifile:
        with open(ofn, 'w', encoding='utf-8') as ofile:
            reader = csv.DictReader(ifile)
            fields = next(reader)
            writer = csv.DictWriter(ofile, fields, lineterminator='\n')
            writer.writeheader()
            for row in reader:
                writer.writerow(standardize_row(row))

# Download

In [33]:
t0 = time.perf_counter()

ses = requests.Session()
finn_codes = fetch_finn_codes(ses, 400, 800)
ads = fetch_ads(ses, finn_codes)
flattened_ads = [flatten(d) for d in ads]
stripped_ads = strip_items(flattened_ads)

t1 = time.perf_counter()
print("Elapsed time: {}".format(t1-t0))

Page 400/800
Page 401/800
Page 402/800
Page 403/800
Page 404/800
Page 405/800
Page 406/800
Page 407/800
Page 408/800
Page 409/800
Page 410/800
Page 411/800
Page 412/800
Page 413/800
Page 414/800
Page 415/800
Page 416/800
Page 417/800
Page 418/800
Page 419/800
Page 420/800
Page 421/800
Page 422/800
Page 423/800
Page 424/800
Page 425/800
Page 426/800
Page 427/800
Page 428/800
Page 429/800
Page 430/800
Page 431/800
Page 432/800
Page 433/800
Page 434/800
Page 435/800
Page 436/800
Page 437/800
Page 438/800
Page 439/800
Page 440/800
Page 441/800
Page 442/800
Page 443/800
Page 444/800
Page 445/800
Page 446/800
Page 447/800
Page 448/800
Page 449/800
Page 450/800
Page 451/800
Page 452/800
Page 453/800
Page 454/800
Page 455/800
Page 456/800
Page 457/800
Page 458/800
Page 459/800
Page 460/800
Page 461/800
Page 462/800
Page 463/800
Page 464/800
Page 465/800
Page 466/800
Page 467/800
Page 468/800
Page 469/800
Page 470/800
Page 471/800
Page 472/800
Page 473/800
Page 474/800
Page 475/800
Page 476/800

97916516
72211848
97916003
97915476
97916407
97915233
97916301
97916169
97916128
97915859
97915957
97915662
97907170
95696217
97913685
97916016
97915856
97915952
97915777
97915847
97915730
97915164
97913916
97915714
97915744
97915709
97915542
97915670
98609492
97915193
97915118
94834643
97915440
97911895
94031872
97915279
97915296
70863244
97915270
97915232
97915165
97914459
97915161
97915133
97914531
97905591
97914403
97912880
74880095
97915012
97915043
97915053
97914146
94995076
97907356
97914892
97775329
97913640
95381665
92793212
95721447
97913751
97914677
97361957
97914613
97914621
97914628
97913925
96730381
97914567
97503578
97914481
88071568
97914408
97913971
97052386
97914057
97913543
97914356
Progress: 408/20400
98591751
75977320
97914335
97913151
97913511
97913669
97913659
97909691
97914015
97914024
76667582
97913947
97909800
97913796
97913747
96158484
97580487
97912803
97913583
97913516
97913316
97912349
97913439
97909042
97911854
91954246
97913259
97913293
97912844
97907080

97869634
97873589
97874162
97874860
97873477
97734261
97873442
97873524
97763717
97872990
90806075
97858962
92409606
97855045
97856025
97872986
97873004
97873612
95807958
97868839
97860492
97872505
97869713
97860625
97871848
97870902
90725556
78754153
97871202
19131920
97872592
97870282
97871337
97871670
97833388
97871713
97868481
97865600
97870357
97870215
97869532
75326613
97869632
97866835
96240765
97871671
98418364
97826740
97864875
97277045
97284473
95662968
97865275
66514502
97871199
97871188
97869379
94743435
86752672
97869279
97869024
97867054
97867506
97865256
97860835
97870015
97869897
97865236
97867853
65968446
97867096
97732141
97866244
97869424
97866488
97868097
97866460
97864904
97858036
85479522
95855270
97868639
97863819
87686130
97865138
97864130
97866125
97867313
95549159
97865967
87621748
97862168
84401699
97861322
97799581
97849631
97500826
97441420
97863961
92685053
97694696
97859989
97864336
97857040
67582224
97861175
97864222
97864278
97857500
97862885
97859084
8

97787273
68719291
82438082
97787658
95310481
95440885
97780212
97788684
92998158
97784482
97787578
95841188
97786305
97785085
97785253
97767037
97783754
97783931
97780312
97787159
97786452
97786289
97784601
97547013
97738872
97786010
97784560
97783792
97785400
97763245
97780975
97770923
73104824
97782360
89116825
97784645
97778494
97783246
97782887
97783918
97782259
97780371
91223216
97780335
97784398
97781746
97783983
97783570
87839378
97778148
97782143
97765427
94521016
91386689
97782267
97781096
97782991
97729808
95600552
97731396
97782552
97732917
98638487
97782152
97783900
97624961
97781996
97780236
97779765
97781048
97783157
96365772
97780454
97779440
97780580
97327551
73936598
97782015
97782358
97771459
88992040
90836712
97780002
97780277
97765679
97781987
97779916
97589447
93172573
97781850
97779345
97776564
97775370
97781629
97779964
97778068
97781335
97781242
97778597
97780290
97779225
97779950
97778040
97750242
97779198
67726970
97778520
97779982
97776712
97779191
90038053
9

97694353
97694394
93316050
97694230
97694210
97267995
97692980
97693857
97693855
93509727
97693640
97692620
97693595
93376506
97692772
97693350
80881408
97690678
97691666
97693238
97693150
97692971
97658917
97692823
92918847
97692840
97692726
97690414
Progress: 3060/20400
87139513
97692423
95739304
97692404
97692328
91856796
97691503
93097720
97692141
97692094
93668221
97691996
97686991
97691973
97691160
97691765
97667300
89683971
97691675
97691012
97691600
89189862
97691386
97675960
97691158
97690241
97691102
97691063
97689942
97691071
97689858
97687653
97671241
97690776
97690380
94726673
96866370
93237283
97688543
97688828
97686089
97688799
97309955
97688990
97689849
97689829
92690553
97689336
97689584
97689533
97688403
96130662
97686798
97689328
97689317
76111997
97687567
97688721
97688834
97688809
97687585
97688282
97688176
95344885
95741153
97687901
92364421
97687742
97687504
93939923
97685055
97687303
97687317
93837912
97686419
91741199
95739507
97680215
97685133
97686628
9758807

...

96857281
96855132
96858933
96846597
96858296
68916731
96857967
87178840
96858949
95618278
96851634
83908505
96801712
94758559
96853092
96856144
66921737
96855053
96854408
96855760
96088631
96854483
94875281
96855629
96854637
94385811
96855608
96854626
96855570
94508278
96855112
93732385
96856225
96855697
96854597
92351132
96849897
96852817
96855260
96852401
94824413
96850039
96854060
93738069
96852187
96853877
97074007
96743763
81936884
96850788
96850050
96850632
93120912
96852592
96850059
96842944
96852123
89711250
96852212
96849377
86954131
96852050
96852078
96850114
96849535
96850027
81949709
96850547
96847664
72706670
77917121
96847685
96850879
96677008
96848798
96848347
96848157
96840118
96848464
96847951
96846598
96849606
96848634
96840775
96847279
94412843
96846673
81033366
96845070
96839679
76555835
96845982
96848643
96842474
96844544
96841744
96846605
80680124
96846393
96845950
96714646
96841757
82490036
96845540
96840522
96843628
96844609
96842625
95609617
96842609
96842764
9

94469295
96744305
96745554
88718604
75038110
96743184
96741320
96745361
96711434
95641581
94870221
98574694
82751135
95237415
96678381
96743661
96743231
96744635
96742368
96743347
96744509
96743238
96739701
96742564
89971776
84758056
96742748
96741830
96740986
96739625
96739629
92287905
96741685
96684618
96741396
91758851
96740772
92942516
86686629
96741317
96739439
94778207
96739270
93169612
96690244
96740501
96741200
96739783
96668650
92580038
92575382
91410842
94633896
94363170
94930412
96738750
96737712
96739306
96738110
96739433
96738896
96739328
96770788
96739049
96738927
96738611
96738574
94783353
92894781
96738373
67670027
94337565
96737733
96735069
93681340
96738031
96733633
96737902
96736783
96734997
71823312
96723060
96731211
96735506
96732673
96735851
96734923
82573971
95916963
96735779
96609916
86144757
96734537
96734575
96732293
96734357
96734175
96730040
96732579
96733173
96731487
96732623
list index out of range
96730705
96728391
94436435
96731814
96732062
96731981
9673

96669550
96669393
96669322
94933884
96669208
96669157
96664158
96668045
96668932
96668353
96668849
96668831
96668848
96668770
96668768
96665312
96668620
list index out of range
96666805
70368320
71110738
96668351
96668251
96668169
96668228
74485737
96667942
86621218
96667762
96666310
96667637
96667530
Progress: 18360/20400
98249988
96667476
96665387
96667389
96667436
96667337
96665377
96666856
96666661
96666579
96666110
96666258
96666166
96666177
96666040
96666025
96665949
96665886
96665941
96665948
96626411
96665833
96665773
96665774
96665792
96665721
96665648
85913592
96665539
96665523
96664858
93838321
96665016
96664946
96662926
96664943
96664957
96664934
96664048
96664806
96664808
96664828
96664754
96664577
81455645
90920560
96664350
96664296
96663991
96663968
96664016
97736271
96663836
93090276
96660704
96663771
96663781
96131140
96663741
96663600
96663538
96661888
96663466
96663451
96663376
96663359
96662281
96660937
96663137
96663076
96662987
96662623
96661620
96663002
96661730


96581853
96581837
96465664
96581828
96581800
96581794
96578131
96581697
96581731
96581704
96581702
96581660
96581654
96580784
96580485
96581570
96581601
96581513
96580915
96581443
96581448
96581399
96581393
96579901
96581292
96581239
96581191
96578683
96581053
96580334
96581004
96580942
96580872
96580875
96580901
96580876
92532301
96580838
96580790
96580716
96580718
96579232
96580697
96580636
96580629
96579364
96580414
96580455
96580400
96580143
96580106
96578773
96579588
98610493
96580117
96578982
96576530
96579994
96579959
96572456
96579759
96579665
96573299
96579622
96579615
96579513
96579544
96579462
96579334
96579368
96578459
96579263
96579046
96579242
93432869
96579112
96578603
96578910
96578900
96578169
96578812
96578796
96578835
81779487
96578765
96578736
96578779
96578674
96578647
96578614
96578582
96578565
96577929
96576537
96578507
74569809
96574819
96578245
96578369
96578291
85825737
96578280
96577166
96578177
98107164
96578171
96578069
87722918
96577993
96577934
96577776
9

96509401
96479128
96507162
96506905
96509141
96507409
76476489
96508848
96509024
96508350
96508810
96508916
96508850
96242280
96508644
84727998
96507208
96508683
96508722
84106602
96508722
96507833
96506411
96507654
96508465
96507769
96506716
96508323
96508256
96508298
96508200
96508099
91996468
96508023
96507973
96507717
96506124
96507662
96507517
96507480
94297947
96507330
96504281
96507005
96506701
96507109
86928516
96504561
96506821
83840095
96506572
96506177
96506463
96504903
96506276
96505965
96506304
96506117
96505793
96505652
96505660
96505453
96505362
96504452
96504752
96504717
96502643
96504523
96504536
96504393
Progress: 20196/20400
98315009
96503443
96503498
96503755
96503681
96501482
96503553
96503432
96503385
96503057
96503078
75210950
96500100
96500417
96502702
96502734
96502603
96502513
96502447
96502364
96502312
96502252
96502281
96502192
96501313
96501967
96501892
96501886
96501797
96501719
96500855
96370409
96467860
90446622
96501531
96383078
96501392
96501279
965011

In [34]:
export_data("output-400-800.csv", stripped_ads)

In [38]:
standardize_csv('output-100.csv', 's-output-100.csv')

Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Ikke oppgitt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse date: 9/6-2011
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Ikke oppgitt
Could not parse date: Mars 2012
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse number: Fritatt
Could not parse date: Juni 2012

# Spare

In [11]:
soup_page = BeautifulSoup(ses.get('http://m.finn.no/car/used/ad.html?finnkode=98543643').text, 'lxml')

In [16]:
data_dl_element = soup_page.find_all('dl')[2]
for key, value in zip(data_dl_element.find_all('dt'), data_dl_element.find_all('dd')):
    print(key, value)

<dt data-automation-id="key">Årsmodell</dt> <dd data-automation-id="value">2001</dd>
<dt data-automation-id="key">1. gang registrert</dt> <dd data-automation-id="value">30.04.2001</dd>
<dt data-automation-id="key">Km.stand</dt> <dd data-automation-id="value">227 500 km</dd>
<dt data-automation-id="key">Farge</dt> <dd data-automation-id="value">Sølv</dd>
<dt data-automation-id="key">Girkasse</dt> <dd data-automation-id="value">Manuell</dd>
<dt data-automation-id="key">Hjuldrift</dt> <dd data-automation-id="value">Forhjulsdrift</dd>
<dt data-automation-id="key">Drivstoff</dt> <dd data-automation-id="value">Bensin</dd>
<dt data-automation-id="key">Effekt</dt> <dd data-automation-id="value">105 Hk</dd>
<dt data-automation-id="key">Sylindervolum</dt> <dd data-automation-id="value">1,6 l</dd>
<dt data-automation-id="key">Vekt</dt> <dd data-automation-id="value">1 215 kg</dd>
<dt data-automation-id="key">CO2 utslipp</dt> <dd data-automation-id="value">168 g/km</dd>
<dt data-automation-id="key

In [19]:
b = soup_page.find_all('dl', attrs={'class': 'r-prl mhn multicol col-count1upto640 col-count2upto768 col-count1upto990 col-count2from990'})